In [ ]:
# todo: figure out how to get saliency for transformer
# can we match CLIP performance with VB?

In [ ]:
from mmf.utils.env import import_user_module
%load_ext autoreload
%autoreload 2
import_user_module("/home/g-luo/remote_home/foil_mmf")
import numpy as np
from PIL import Image
import torch
from torchray.attribution.grad_cam import grad_cam
from miniclip.imageWrangle import heatmap, heatmap_helper, min_max_norm, torch_to_rgba
import matplotlib.pyplot as plt
import json

def get_feat(ann):
    base = "/shared/g-luo/visual_news_metadata/features/"
    rest = ann["image_path"].replace("./", "").replace(".jpg", "_info.npy").replace("images", "features")
    return base + rest

def get_categories():
    categories_unprocessed = json.load(open("../foil_mmf/etc/visual_genome_categories.json"))
    categories = {cat["id"]: cat["name"] for cat in categories_unprocessed["categories"]}
    return categories

device = "cuda" if torch.cuda.is_available() else "cpu"
label_categories = get_categories()

In [ ]:
import sys
# sys.path.append("../foil_mmf/foil/models")
from natural_language_joint_query_search.CLIP import clip

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"
rn50_dir = "/shared/g-luo/tarasque/emnlp/RN50_freeze_lower/merged_balanced_clip_rn50_freeze_lower/"

In [ ]:
predictions = json.load(open("../foil_mmf/val/merged_balanced_clip_rn50_freeze_lower/foil_clip_2/reports/val.json"))
predictions = {ann["id"]:ann for ann in predictions}

In [ ]:
import torch
from collections import OrderedDict

device = "cuda"
def get_model(load_weights=True, model_dir=""):
    model, preprocess =  clip.load("RN50", device=device, jit=False)
    if load_weights:
        print("Loaded")
        state_dict = torch.load(f"{model_dir}")
        lst = []
        for key in state_dict["model"]:
            new_key = key.replace("model.", "")
            if "classifier" not in new_key:
                lst.append((new_key, state_dict["model"][key]))
        model.load_state_dict(OrderedDict(lst))
        model.to("cuda")
        model.eval()
    return model, preprocess

finetuned_model, preprocess = get_model(load_weights=True, model_dir=rn50_dir + "best.ckpt")
finetuned_model = finetuned_model.float()

original_model, _ = get_model(load_weights=False)
original_model = original_model.float()

In [ ]:
# from foil_mmf.foil.models.clip import CLIP
# from mmf.utils.configuration import load_yaml_with_defaults

# conf = load_yaml_with_defaults(rn50_dir + "config.yaml")
# classification_model = CLIP(conf.model_config.clip)
# classification_model.build()

# state_dict = torch.load(rn50_dir + "/best.ckpt")
# classification_model.load_state_dict(state_dict["model"])
# classification_model.to("cuda")
# classification_model.eval()

In [ ]:
import json
import numpy as np
option = "merged/balanced_disjoint_ents_quad"
dataset_type = "val"
dataset = json.load(open(f"/home/g-luo/.cache/torch/mmf/data/datasets/foil/defaults/annotations/visualnews_foil/{option}/{dataset_type}.json"))["annotations"]
metadata = json.load(open(f"/shared/g-luo/visual_news_metadata/visualnews_foil/full_split/data/{dataset_type}.json"))

In [ ]:
ents = {ann["id"]:ann["caption_entities"] for ann in metadata}

In [ ]:
# https://github.com/openai/CLIP/issues/18
# TODO: figure out saliency for ViT-B32

In [ ]:
alpha = [0.5, 0.7, 0.8][0]
layer = 'layer4.2.relu'
def get_grad_cam(model, image, text):
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features, _ = model.encode_text(text)
        image_features_norm = image_features.norm(dim=-1, keepdim=True)
        image_features_new = image_features / image_features_norm
        text_features_norm = text_features.norm(dim=-1, keepdim=True)
        text_features_new = text_features / text_features_norm
        logit_scale = model.logit_scale.exp()
        logits_per_image = logit_scale * image_features_new @ text_features_new.t()
        probs = logits_per_image.softmax(dim=-1).cpu().numpy().tolist()

    saliency = grad_cam(model.visual, image.type(model.dtype), image_features, saliency_layer=layer)
    hm = heatmap(image[0], saliency[0][0,].detach().type(torch.float32).cpu(), alpha=alpha)

    collect_images, saliencies = [], []
    for i in range(len(categories)):
        # mutliply the normalized text embedding with image norm to get approx image embedding
        text_prediction = (text_features_new[[i]] * image_features_norm)
        saliency = grad_cam(model.visual, image.type(model.dtype), text_prediction, saliency_layer=layer)
        hm_helper = heatmap_helper(image[0], saliency[0][0,].detach().type(torch.float32).cpu(), alpha=alpha)
        hm = heatmap(image[0], saliency[0][0,].detach().type(torch.float32).cpu(), alpha=alpha)
        collect_images.append(hm)
        saliencies.append(hm_helper)
    logits = logits_per_image.cpu().numpy().tolist()[0]
    titles = [f"{x} - {str(round(y, 3))}/{str(round(l, 2))}" for (x, y, l) in zip(categories, probs[0], logits)]
    return titles, collect_images, saliencies

In [ ]:
import matplotlib.pyplot as plt 
from PIL import Image
import matplotlib.patches as patches
import torch, os

# get the proportion of times that the attn lands in each bbox
def get_attention_bbox(ann, saliency, threshold=0.5):
    saliency = saliency / np.sum(saliency)
    img_info = np.load(get_feat(ann), allow_pickle=True).item()
    img_path = "/shared/g-luo/visual_news/origin/" + ann["image_path"].split("./")[1]
    img = Image.open(img_path)
    img_array = torch_to_rgba(image[0])

    # bboxes have no need for normalization
    weights = []
    for i in range(100):
        if img_info["cls_prob"][i].max() < threshold:
            continue
        w, h = img_info["image_width"], img_info["image_height"]
        x1, y1, x2, y2 = img_info["bbox"][i]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        label_idx = np.argmax(img_info["cls_prob"][i])
        label = label_categories[label_idx]
        
        weights.append((label, np.sum(saliency[x1:x2, y1:y2])))
    print(max(weights, key=lambda x:x[1]))

In [ ]:
# Taken from https://github.com/pytorch/captum/blob/master/captum/attr/_utils/visualization.py
from IPython.core.display import HTML, display
def _get_color(attr):
    # clip values to prevent CSS errors (Values should be from [-1,1])
    attr = max(-1, min(1, attr))
    if attr > 0:
        hue = 120
        sat = 75
        lig = 100 - int(50 * attr)
    else:
        hue = 0
        sat = 75
        lig = 100 - int(-40 * attr)
    return "hsl({}, {}%, {}%)".format(hue, sat, lig)

def format_special_tokens(token):
    if token.startswith("<") and token.endswith(">"):
        return "#" + token.strip("<>")
    return token

def format_word_importances(words, importances):
    if importances is None or len(importances) == 0:
        return "<td></td>"
    assert len(words) <= len(importances)
    tags = ["<td>"]
    for word, importance in zip(words, importances[: len(words)]):
        word = format_special_tokens(word)
        color = _get_color(importance)
        unwrapped_tag = '<mark style="background-color: {color}; opacity:1.0; \
                    line-height:1.75"><font color="black"> {word}\
                    </font></mark>'.format(
            color=color, word=word
        )
        tags.append(unwrapped_tag)
    tags.append("</td>")
    return "".join(tags)

def visualize_text(
    datarecords, title
) -> "HTML":  # In quotes because this type doesn't exist in standalone mode
    dom = ["<table width: 100%>"]
    rows = [
        "<th>" + title + "</th>"
    ]
    for datarecord in datarecords:
        rows.append(
            "".join(
                [
                    "<tr>",
                    format_word_importances(
                        datarecord.raw_input, datarecord.word_attributions
                    ),
                    "<tr>",
                ]
            )
        )

    dom.append("".join(rows))
    dom.append("</table>")
    html = HTML("".join(dom))
    display(html)

    return html

In [ ]:
import seaborn as sns
from captum.attr import visualization
def get_text_scores(model, image, ann, title):
    words = ann["caption"].split(" ")
    text = clip.tokenize(ann["caption"]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features, weight = model.encode_text(text)
        
    attention_weights = list(weight[-1][0][1+len(words)].cpu().numpy())[:2+len(words)][1:][:-1]
    attention_weights = [float(item) for item in attention_weights]
#     scores = (image_features @ text_features.T).cpu().numpy()[0]
#     scores = scores / np.sum(scores)
#     #words = np.expand_dims(np.array(words), 0)
#     print(scores)
    vis_data_records = [visualization.VisualizationDataRecord(attention_weights,0,0,0,0,0,words,1)]
    visualize_text(vis_data_records, title)

In [ ]:
for i in range(20, 50):
    ann = dataset[i]
    print(ann["id"])
    print("Caption: ", ann["caption"])
    print("Image Caption: ", ann["image_caption"])
    print("")
    base_path = "/shared/g-luo/visual_news/origin/"
    imageFile = base_path + ann["image_path"].replace("./", "")
#     categories = [f"an image of {ent[0]}" for ent in ents[ann["id"]]]
    categories = [ann["caption"]]
    if imageFile:
        image_raw = Image.open(imageFile)
        # preprocess image:
        image = preprocess(image_raw).unsqueeze(0).to(device)
        # preprocess text
        # categories = [ann["caption"]]
        text = clip.tokenize(categories).to(device)
        
        get_text_scores(finetuned_model, image, ann, "Finetuned Word Importance")
        get_text_scores(original_model, image, ann, "Original Word Importance")

        finetuned_titles, finetuned_collect_images, finetuned_saliencies = get_grad_cam(finetuned_model, image, text)
        original_titles, original_collect_images, original_saliences = get_grad_cam(original_model, image, text)
        
        if i % 2 == 0:
            predicted = predictions[ann["id"]]
        else: 
            predicted = predictions[ann["id"] * -1]
            
        for i in range(len(finetuned_collect_images)):
            f, ax = plt.subplots(1,2)
            # print("FINETUNED, FREEZE LOWER ", finetuned_titles[i])
            print("Target ", predicted["target"], "Prediction ", predicted["score"])
            ax[0].imshow(finetuned_collect_images[i])
            print(" ")
            # print("ORIGINAL ",original_titles[i])
            ax[1].imshow(original_collect_images[i])
            plt.show()
            
        # get_attention_bbox(ann, finetuned_saliencies[0])
        print("===================================")

    #     print("Original Image and Grad Cam for image embedding")
    #     original, gradcam = Image.fromarray((torch_to_rgba(image[0]).numpy() * 255.).astype(np.uint8)), hm
    #     fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))
    #     axes[0].imshow(original)
    #     axes[1].imshow(gradcam)
    #     plt.show()